В данном блоке начинается процесс по приведению датасета в вид, пригодный для обучения модели

<img src='./imgs/MasterDeck - Inferece.jpg' alt='Схема обучения' width='900'>
</img>

Основная того, как модель будет работать по итогу - мы передаем в модель не только изначальный текст, но так же и дополнительную информаци:
* Класс токсичности, опрделенный сторонней моделью
* Класс эмоциональности, определенный сторонней моделью
* Информация о продукте (Описание товара, магазин, категория, это Отзыв или Вопрос и тд.)

<img src='./imgs/MasterDeck%20-%20Train.jpg' alt='Схема обучения' width='900'>
</img>

Для того, чтоюы модель работала по такому механизму её нужно дообучить на корпусе, состоящем из изначального текста и текста, который мы хотим получить в итоге

Так как у нас есть просто набор текстов, нужно изначально выделить "Плохие" тексты (токсичные, негативные излишне депрессивные), которые мы будем менять, и "Хорошие" тексты (в основном нейтральные, но могут быть и позитивными), которые будут для нас таргетом

После этого мы должны найти ближайшие тексты по смыслу из группы "Хороших" на каждый "Плохой" текст, с помошью матчинга

In [30]:
import random
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForSequenceClassification
from sentence_transformers import SentenceTransformer

Классификация по эмоциональности и токсичности, понадобиться дальше для формирование промта и подбора обучающей выборки

\*Решил протестировать скорость работы optimum'а с Onnx в бекенде

In [2]:
def load_calssif(name: str):
    return pipeline(
        task='text-classification',
        model=ORTModelForSequenceClassification.from_pretrained(
            name,
            export=True,
            provider="CUDAExecutionProvider",
            use_io_binding=True
        ), # type: ignore
        tokenizer=AutoTokenizer.from_pretrained(
            name,
            useFast=True
        ),
        device='cuda:0'
    )

In [6]:
ds = load_dataset('Roaoch/urfu_ecom_wb', split='train')

ds

Dataset({
    features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', '__index_level_0__'],
    num_rows: 1914329
})

In [7]:
ds = ds.train_test_split( # type: ignore
    test_size=0.25,
    shuffle=True
)

ds

DatasetDict({
    train: Dataset({
        features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', '__index_level_0__'],
        num_rows: 1435746
    })
    test: Dataset({
        features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', '__index_level_0__'],
        num_rows: 478583
    })
})

In [8]:
tox_class_pipe = load_calssif('cointegrated/rubert-tiny-toxicity')

config.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
emot_class_pipe = load_calssif('cointegrated/rubert-tiny2-cedr-emotion-detection')

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Для матчинга нам нужно вытянуть вектор предложения, для чего буду использовать Sentence-transformers, так же с бекендом в виде Onnx

In [10]:
vecorizer = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    backend='onnx'
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

*************** EP Error ***************
EP Error D:\a\_work\1\s\onnxruntime\python\onnxruntime_pybind_state.cc:507 onnxruntime::python::RegisterTensorRTPluginsAsCustomOps Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************


tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Обрабатываем тренировочные тексты моделям, тестовый набор оставляем для валидации в конце

In [19]:
def predict(row):
    return {
        **row,
        'vector': vecorizer.encode(row['text'][:vecorizer.max_seq_length]),
        'toxicity': tox_class_pipe(row['text'][:tox_class_pipe.tokenizer.model_max_length])[0]['label'], # type: ignore
        'emotions': emot_class_pipe(row['text'][:emot_class_pipe.tokenizer.model_max_length])[0]['label'] # type: ignore
    }


ds['train'] = ds['train'].map(
    predict
)

ds

Map:   0%|          | 0/1435746 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', '__index_level_0__', 'vector', 'toxicity', 'emotions'],
        num_rows: 1435746
    })
    test: Dataset({
        features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', '__index_level_0__'],
        num_rows: 478583
    })
})

In [26]:
ds['train'].to_parquet('./__output__/train.parquet')
ds['test'].to_parquet('./__output__/test.parquet')

Creating parquet from Arrow format:   0%|          | 0/1436 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/479 [00:00<?, ?ba/s]

846985890

In [44]:
ds['train'][random.randint(0, len(ds['train']))]

{'nmId': 15250124,
 'text': 'почему за ваш брак с меня ещё содрали 300 р за возврат??? проверила на пункте выдачи открыв посылку в присутствие оператора и там трещина на колбе!!! у вас не качественный товар с браком и вы бессовестные с меня ещё удержал 300р, потом ещё кому-нибудь отправите и так по кругу будете людей обманывать. вот так значит валберес зарабатывает....',
 'responder': 'Клиент',
 'type': 'Отзыв',
 'product_name': 'Погружной мощный блендер с насадками 700 Вт , SC-HB42F91',
 'product_category_2': 'Блендеры',
 'product_category_1': 'Техника для кухни',
 'product_color': '',
 'product_description': 'Погружной блендер Scarlett SC-HB42F91 сочетает в себе функционал блендера, миксера и измельчителя. Утонченный белый корпус небольшого веса и мощный мотор созданы специально для удобной работы и компактного хранения даже на самой маленькой кухне. 14 плавно переключающихся скоростных режимов дают свободу выбора и полет фантазии для приготовления самых изысканных блюд на каждый ден